In [27]:
import twitter
import time
import os
import csv

cons_key = "mNxnoZIpZYeHJV9GaM4Rz0Cwi"
cons_sec = "ooMKnp4ozpAaYE0eZpuFMcyqSqYiMolGwSMJmM97Cf3En599AA"
acc_key = "1180536763346894849-KD5UvELJbFKrshap2yjNM0aQjGKn2v"
acc_sec = "aoiBD4MUiM2136WZweraKvLac0qITelSTcmKhXSl4WGyE"

# initialize api instance
twitter_api = twitter.Api(consumer_key=cons_key,
                        consumer_secret=cons_sec,
                        access_token_key=acc_key,
                        access_token_secret=acc_sec)

# test authentication
print(twitter_api.VerifyCredentials())

{"created_at": "Sat Oct 05 17:34:40 +0000 2019", "default_profile": true, "description": "Dev Account XD", "id": 1180536763346894849, "id_str": "1180536763346894849", "name": "99PercentBeast", "profile_background_color": "F5F8FA", "profile_image_url": "http://pbs.twimg.com/profile_images/1180536857290932224/SmY5M2ff_normal.jpg", "profile_image_url_https": "https://pbs.twimg.com/profile_images/1180536857290932224/SmY5M2ff_normal.jpg", "profile_link_color": "1DA1F2", "profile_sidebar_border_color": "C0DEED", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "profile_use_background_image": true, "screen_name": "beast_percent", "status": {"created_at": "Sun Oct 06 06:28:26 +0000 2019", "id": 1180731521549758465, "id_str": "1180731521549758465", "lang": "en", "source": "<a href=\"https://epic-sentiment-analysis.com\" rel=\"nofollow\">epic-sentiment-analysis</a>", "text": "This is post update on twitter"}, "statuses_count": 1}


In [2]:
def buildTestSet(search_keyword):
    try:
        tweets_fetched = twitter_api.GetSearch(search_keyword, count = 100)
        
        print("Fetched " + str(len(tweets_fetched)) + " tweets for the term " + search_keyword)
        
        return [{"text":status.text, "label":None} for status in tweets_fetched]
    except:
        print("Unfortunately, something went wrong..")
        return None

In [4]:
search_term = input("Enter a search keyword:")
testDataSet = buildTestSet(search_term)

print(testDataSet[0:4])

Enter a search keyword: meme


Fetched 100 tweets for the term meme
[{'text': "C'est grave quand même... #ASCOM #teamOM\n(images @CanalplusSport) https://t.co/wBGdPIgJs0", 'label': None}, {'text': 'Speaker Nancy Pelosi: "There\'s this meme going around of me and I\'m saying, \'Donald, you used to own a casino. You… https://t.co/Ppo6KdwNEW', 'label': None}, {'text': 'LaRouche campaign plants deranged questioner who asks @AOC if we should “eat our babies” to protect the environment… https://t.co/2L7Lv89A6g', 'label': None}, {'text': 'RT @TvHenni7: @Kulturlesite_ Ptdr oula jai cru voir gambi et travis scott dans la meme phrase je suis fatigué jv me recoucher', 'label': None}]


In [6]:
##status = twitter_api.PostUpdate("This is post update on twitter")
##print(status)

In [28]:
import pandas as pd
corpus = pd.read_csv("data/corpus.csv",header=-1)
columns = ["topic","label","tweet_id"]
corpus.columns = columns

In [29]:
trainingdata = []
type(trainingdata)

list

In [56]:
def build_training_set(corpusfile):
    for index, tweet in corpusfile.iterrows():
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print(status.text)
            tweet["text"] = status.text
            trainingdata.append(tweet)
            #print(tweet)
            print(trainingdata)
        except:
            continue
    return(trainingdata)

In [ ]:
trainingdata = build_training_set(corpus[:10])

In [53]:
trainingdata = pd.DataFrame(trainingdata)
trainingdata.head()

,topic,label,tweet_id
0,apple,positive,126415614616154112
0,apple,positive,126415614616154112
2,apple,positive,126402758403305474
3,apple,positive,126397179614068736
6,apple,positive,126379685453119488


In [48]:
trainingdata.to_csv("data/trainingdata.csv",index=False)

In [49]:
ts = pd.read_csv("data/trainingdata.csv")

In [50]:
ts.head()

,topic,label,tweet_id
0,apple,positive,126415614616154112
1,apple,positive,126415614616154112
2,apple,positive,126402758403305474
3,apple,positive,126397179614068736
4,apple,positive,126379685453119488


In [11]:
## creating training data set around 5k tweets
def build_training_set(corpusfile,full_corpus):
    ## create array with variable name corpus
    corpus = []
    ## tweet's id is avaliable in our corpus csv file with sentiment
    with open(corpusfile,'rt') as csvfile:
        ## reading our csv file
        lineread = csv.reader(csvfile,delimiter=',',quotechar="\"")
        for row in lineread:
            corpus.append({
            "tweet_id":row[2], # tweet's id 
            "label":row[1], ## sentiment
            "topic":row[0]
            }) ## topic related to it
    
    limit = 150
    sleeptime = 900/limit
    
    
    trainingdata = []        
    for tweet in corpus:
        try:
            status = twitter_api.GetStatus(tweet["tweet_id"])
            print("Tweet is : ",status.text)
            tweet["text"] = status.text
            trainingdata.append(tweet)
            time.sleep(sleeptime)
        except:
            continue
            
    with open(full_corpus,'rwb') as tdf:
        line = csv.writer(tdf,delimiter=",",quotechar="\"")
        for row in trainingdata:
            try:
                line.writerow(row['tweet_id'],row['text'],row['label'],row['topic'])
            except:
                print(e)
    return(trainingdata)   

In [12]:
corpusfile = "data/corpus.csv"
full_corpus = "data/trainingdata.csv"

In [ ]:
build_training_set(corpusfile,full_corpus)